In [1]:
import time
import os

import pandas as pd
import numpy as np
np.set_printoptions(precision=6, suppress=True)

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import *
tf.__version__

'2.3.0'

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 2 Logical GPUs


In [3]:
from tensorflow.keras.metrics import Metric
class RSquare(Metric):
    """Compute R^2 score.
     This is also called as coefficient of determination.
     It tells how close are data to the fitted regression line.
     - Highest score can be 1.0 and it indicates that the predictors
       perfectly accounts for variation in the target.
     - Score 0.0 indicates that the predictors do not
       account for variation in the target.
     - It can also be negative if the model is worse.
     Usage:
     ```python
     actuals = tf.constant([1, 4, 3], dtype=tf.float32)
     preds = tf.constant([2, 4, 4], dtype=tf.float32)
     result = tf.keras.metrics.RSquare()
     result.update_state(actuals, preds)
     print('R^2 score is: ', r1.result().numpy()) # 0.57142866
    ```
    """

    def __init__(self, name='r_square', dtype=tf.float32):
        super(RSquare, self).__init__(name=name, dtype=dtype)
        self.squared_sum = self.add_weight("squared_sum", initializer="zeros")
        self.sum = self.add_weight("sum", initializer="zeros")
        self.res = self.add_weight("residual", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred):
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        self.squared_sum.assign_add(tf.reduce_sum(y_true**2))
        self.sum.assign_add(tf.reduce_sum(y_true))
        self.res.assign_add(
            tf.reduce_sum(tf.square(tf.subtract(y_true, y_pred))))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        mean = self.sum / self.count
        total = self.squared_sum - 2 * self.sum * mean + self.count * mean**2
        return 1 - (self.res / total)

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.squared_sum.assign(0.0)
        self.sum.assign(0.0)
        self.res.assign(0.0)
        self.count.assign(0.0)

In [4]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
plt.rcParams['figure.figsize'] = ((8/2.54), (6/2.54))
plt.rcParams["font.family"] = "Arial"
plt.rcParams["mathtext.default"] = "rm"
plt.rcParams.update({'font.size': 11})
MARKER_SIZE = 15
cmap_m = ["#f4a6ad", "#f6957e", "#fccfa2", "#8de7be", "#86d6f2", "#24a9e4", "#b586e0", "#d7f293"]
cmap = ["#e94d5b", "#ef4d28", "#f9a54f", "#25b575", "#1bb1e7", "#1477a2", "#a662e5", "#c2f442"]

plt.rcParams['axes.spines.top'] = False
# plt.rcParams['axes.edgecolor'] = 
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.width'] = 1

# Mining

## hyperparameters

In [6]:
SIZE = 10
LOSS_RATES = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
DISP_STEPS = 100
TRAINING_EPOCHS = 500
BATCH_SIZE = 32
LEARNING_RATE = 0.001

In [7]:
class FFNN(Model):
    def __init__(self, out_len):
        super(FFNN, self).__init__()
        
        self.out_len = out_len
        self.dense1 = layers.Dense(64)
        self.bn1 = layers.BatchNormalization()
        self.dense2 = layers.Dense(64)
        self.out_layer = layers.Dense(self.out_len)
        
    def call(self, inp):
        output = self.dense1(inp)
        output = self.bn1(inp)
        output = self.dense2(inp)
        output = self.out_layer(inp)
        
        return output

In [8]:
#loss inputs should be masked.
loss_object = tf.keras.losses.MeanSquaredError()
def loss_function(model, inp, tar): #RNN specialized
    
    masked_real = tar * (1 - inp[..., 5:10])
    masked_pred = model(inp) * (1 - inp[..., 5:10])
    
    return loss_object(masked_real, masked_pred)

## data loading

In [9]:
for LOSS_RATE in LOSS_RATES:
    
    l = np.load('./data/tot_dataset_FFNN_loss_%.2f.npz' % LOSS_RATE)
    raw_input = l['raw_input']
    raw_label = l['raw_label']
    test_input = l['test_input']
    test_label = l['test_label']
    MAXS = l['MAXS']
    MINS = l['MINS']
    MAXS = MAXS[:5]
    MINS = MINS[:5]
    
    raw_input = raw_input.astype(np.float32)
    raw_label = raw_label.astype(np.float32)
    test_input = test_input.astype(np.float32)
    test_label = test_label.astype(np.float32)

    num_train = int(raw_input.shape[0]*.7)
    raw_input, raw_label = shuffle(raw_input, raw_label, random_state=4574)
    train_input, train_label = raw_input[:num_train, ...], raw_label[:num_train, ...]
    val_input, val_label = raw_input[num_train:, ...], raw_label[num_train:, ...]

    train_dataset = tf.data.Dataset.from_tensor_slices((train_input, train_label))
    train_dataset = train_dataset.cache().shuffle(BATCH_SIZE*50).batch(BATCH_SIZE)
    val_dataset = tf.data.Dataset.from_tensor_slices((val_input, val_label))
    val_dataset = val_dataset.cache().shuffle(BATCH_SIZE*50).batch(BATCH_SIZE)
    test_dataset = tf.data.Dataset.from_tensor_slices((test_input, test_label))
    test_dataset = test_dataset.batch(BATCH_SIZE)

    ffnn_model = FFNN(test_label.shape[-1])
    opt = tf.optimizers.Adam(learning_rate=LEARNING_RATE)

    print('Training for loss rate %.2f start.' % LOSS_RATE)
    BEST_PATH = './checkpoints/FFNN_best_loss_%.2fp' % LOSS_RATE

    @tf.function
    def train(loss_function, model, opt, inp, tar):
        with tf.GradientTape() as tape:
            gradients = tape.gradient(loss_function(model, inp, tar), model.trainable_variables)
            gradient_variables = zip(gradients, model.trainable_variables)
            opt.apply_gradients(gradient_variables)

    checkpoint_path = BEST_PATH

    ckpt = tf.train.Checkpoint(ffnn_model=ffnn_model,
                               opt=opt)

    ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=10)

    writer = tf.summary.create_file_writer('tmp')


    prev_test_loss = 100.0
    early_stop_buffer = 500
    with writer.as_default():
        with tf.summary.record_if(True):
            for epoch in range(TRAINING_EPOCHS):
                for step, (inp, tar) in enumerate(train_dataset):
                    train(loss_function, ffnn_model, opt, inp, tar)
                    loss_values = loss_function(ffnn_model, inp, tar)
                    tf.summary.scalar('loss', loss_values, step=step)

                    if step % DISP_STEPS == 0:
                        test_loss = 0
                        for step_, (inp_, tar_) in enumerate(test_dataset):
                            test_loss += loss_function(ffnn_model, inp_, tar_)

                            if step_ > DISP_STEPS:
                                test_loss /= DISP_STEPS
                                break
                        if test_loss.numpy() < prev_test_loss:
                            ckpt_save_path = ckpt_manager.save()
                            prev_test_loss = test_loss.numpy()
                            print('Saving checkpoint at {}'.format(ckpt_save_path))
                        else:
                            early_stop_buffer -= 1

                        print('Epoch {} batch {} train loss: {:.4f} test loss: {:.4f}'
                              .format(epoch, step, loss_values.numpy(), test_loss.numpy()))
                    if early_stop_buffer <= 0:
                        print('early stop.')
                        break
                if early_stop_buffer <= 0:
                    break

    i = -1
    if ckpt_manager.checkpoints:
        ckpt.restore(ckpt_manager.checkpoints[i])
        print ('Checkpoint ' + ckpt_manager.checkpoints[i][-2:] +' restored!!')

    ffnn_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                       loss = tf.keras.losses.MeanSquaredError())

    pred_result = ffnn_model.predict(test_dataset)
    masking = test_input[..., 5:10]
    masked_pred = np.ma.array(pred_result, mask=masking)
    masked_label = np.ma.array(test_label, mask=masking)

    plot_label = ((MAXS[:5]-MINS[:5])*masked_label[..., :5] + MINS[:5])
    plot_label.fill_value = np.nan
    plot_pred = ((MAXS[:5]-MINS[:5])*masked_pred[..., :5] + MINS[:5])
    plot_pred.fill_value = np.nan

    f = open('./results/FFNN_%.2fp.npz' % LOSS_RATE, 'wb')
    np.savez(f,
             test_label = plot_label.filled(),
             test_pred = plot_pred.filled()
            )
    f.close()

Training for loss rate 0.10 start.
Saving checkpoint at ./checkpoints/FFNN_best_loss_0.10p/ckpt-1
Epoch 0 batch 0 train loss: 0.3293 test loss: 0.3585
Saving checkpoint at ./checkpoints/FFNN_best_loss_0.10p/ckpt-2
Epoch 0 batch 100 train loss: 0.0500 test loss: 0.1298
Saving checkpoint at ./checkpoints/FFNN_best_loss_0.10p/ckpt-3
Epoch 0 batch 200 train loss: 0.0573 test loss: 0.0746
Saving checkpoint at ./checkpoints/FFNN_best_loss_0.10p/ckpt-4
Epoch 0 batch 300 train loss: 0.0625 test loss: 0.0577
Saving checkpoint at ./checkpoints/FFNN_best_loss_0.10p/ckpt-5
Epoch 0 batch 400 train loss: 0.0229 test loss: 0.0488
Saving checkpoint at ./checkpoints/FFNN_best_loss_0.10p/ckpt-6
Epoch 0 batch 500 train loss: 0.0131 test loss: 0.0418
Saving checkpoint at ./checkpoints/FFNN_best_loss_0.10p/ckpt-7
Epoch 0 batch 600 train loss: 0.0314 test loss: 0.0354
Saving checkpoint at ./checkpoints/FFNN_best_loss_0.10p/ckpt-8
Epoch 0 batch 700 train loss: 0.0110 test loss: 0.0311
Saving checkpoint at ./